In [126]:
import pandas as pd
import numpy as np

In [127]:
has_parent = pd.read_csv('2022bkk-buget-has_parent.csv')
item = pd.read_csv('2022bkk-buget-item.csv', index_col='id')

In [128]:
df = item.join(has_parent.set_index('id'), on='id')

In [129]:
df.amount = df.amount.astype(np.float128)

In [130]:

psum = df.groupby('parent').amount.sum()
psumd = df.loc[df.parent.drop(df[df.parent == 0].index).unique()].amount.agg(lambda x: x - psum.loc[x.index])
psumb = df.loc[df.parent.drop(df[df.parent == 0].index).unique()].amount.agg(lambda x: x == psum.loc[x.index])

In [131]:
df_ = df.assign(
  is_equal=df.apply(lambda x: psumb.loc[x.parent if x.parent != 0 else x.name], axis=1),
  diff=df.apply(lambda x: psumd.loc[x.parent if x.parent != 0 else x.name], axis=1))

In [132]:
df_.to_csv('2022bkk-buget-item_with_parent.csv')

In [133]:
tf = df_[df_.is_equal == False]
bkkclean = pd.read_csv('bkk-bud_cleaned.csv', index_col='index')


In [134]:
len(tf)/len(df)

0.010752307855551233

In [135]:
bkkclean.groupby(['filename', 'page', 'line_num']).groups.__len__(),tf.groupby(['filename', 'page', 'line_num']).groups.__len__()

(43112, 304)

In [136]:
tf[['filename', 'page', 'line_num', 'text', 'amount',  'parent', 'is_equal','diff']].to_csv('toFix.csv')

In [137]:

idx = []
bkkcleang = bkkclean.groupby(['filename', 'page', 'line_num'])
for x, g in tf.groupby(['filename', 'page', 'line_num']):
  idx += bkkcleang.get_group(x).index.tolist()

In [138]:
bkkclean.loc[idx].to_csv('bkk-toFix.csv')

In [139]:
# pd.read_csv('bkk-toFix.csv', index_col='index').to_dict('records')